# Nettoyage du jeu de données

Ce jeu de données contient plus de 7,2 millions d'articles vendus. Cependant après analyse on se rend compte qu'il y a un travail de recatégorisation concernant les familles, les univers et les mailles.

La colonne maille correspond en fait à la colonne univers. Certains univers et mailles sont trop restreints et il est nécessaire de généraliser et de concaténer certaines catégories.

De plus certaines mailles sont dans le mauvais univers. 

Le but de ce notebook est de faire ressortir un jeu de données propres avec les articles bien catégorisés.

## Importation du jeu de données


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_rows', 200)

In [3]:
df = pd.read_csv('KaDo.csv')

## Nettoyage de la colonne FAMILLE

### Suppression des catégories MULTI FAMILLES et SANTE NATURELLE contenant uniquement 31 articles. 

In [4]:
df = df[(df.FAMILLE != 'MULTI FAMILLES') & (df.FAMILLE != 'SANTE NATURELLE')]

In [5]:
df['FAMILLE'].value_counts()

HYGIENE            2095077
MAQUILLAGE         1694751
SOINS DU VISAGE    1494034
SOINS DU CORPS      878923
PARFUMAGE           628087
CAPILLAIRES         301365
SOLAIRES            153254
Name: FAMILLE, dtype: int64

### Inversion des noms des colonnes Maille et Univers et repositionnement des colonnes dans l'ordre FAMILLE UNIVERS MAILLE

In [6]:
df = df.rename({'UNIVERS': 'MAILLE', 'MAILLE': 'UNIVERS'}, axis="columns")

In [7]:
def string_cleaner(x):
    split_maille = x.split('_')
    if (len(split_maille) > 1):
        maille = ' '.join(split_maille[1:])
    else:
        maille = split_maille[0]
    return maille

In [8]:
df['UNIVERS'] = df['UNIVERS'].apply(string_cleaner)

In [9]:
df['MAILLE'] = df['MAILLE'].apply(string_cleaner)

In [10]:
# Repositionne la colonne UNIVERS avant la colonne MAILLE
df = df.reindex(['TICKET_ID', 'MOIS_VENTE', 'PRIX_NET', 'FAMILLE', 'UNIVERS', 'MAILLE',
       'LIBELLE', 'CLI_ID'], axis=1)

### Renommage de certaines catégories et concaténation d'univers similaires

In [11]:
univers_map = {
    'JDM': 'JARDINMONDE',
    'JEUNE ET LEVRE':'JEUNES',
    'PLAISIRNAT BAIN SAVON': 'PLAISIRNATURE',
    'HYDRA NOURRI ET SOINS':'HYDRATANT NOURRISSANT SOINS',
    'HYDR LAIT HUILE': 'LAIT HUILE',
    'YEUX CLASSIQUE': 'YEUX',
    'YEUX MASCA EYEL FARD': 'YEUX', 
    'AAAR HORS DEMAQLOTION':'ANTI-AGE ANTI-RIDES',
    'MONOI ET EDIT SPEC': 'MONOI',
    'MONOI': 'MONOI',
    'EDT': 'EAU DE TOILETTE',
    'HOMME':'HOMMES',
    'LEV RAL HMG':'LEVRES',
    'LEV BASPRIX':'LEVRES', 
    'AAAR DEMAQLOTION':'ANTI-AGE ANTI-RIDES',
    'CULTUREBIO':'BIO',
    'YEUX MASCA HG':'YEUX',
    'PUR': 'NETTOYANT',
    'AUTRE': 'CHEVEUX'
}

In [12]:
df['UNIVERS'].replace(univers_map, inplace=True)

In [13]:
univers = df['UNIVERS'].value_counts().sort_index().index
for uni in univers:
    print(uni)
    print('-----')
    print(df[df['UNIVERS'] == uni]['MAILLE'].value_counts())
    print('    ')

ANTI-AGE ANTI-RIDES
-----
CJOUR AAAR      182740
DEMAQ AAAR       77767
CNUIT AAAR       51388
TRAIT AAAR       48151
LOTIONS AAAR     15415
MASQUE AAAR       3348
Name: MAILLE, dtype: int64
    
AUTRES
-----
DEMAQ BLEUET            175956
DOUCHE HAMMAM            30611
DOUCHE FRAICHEUR VEG     15180
DOUCHE AUTRES            10314
CNUIT Autres                20
LOTIONS Autres              12
DEO AUTRES                   8
Name: MAILLE, dtype: int64
    
BIO
-----
DOUCHE CULTUREBIO    51642
DEO CULTURE BIO      11549
CJOUR BIO             6460
DEMAQ BIO             2255
CNUIT BIO              394
TRAIT BIO              325
LOTIONS BIO              1
Name: MAILLE, dtype: int64
    
CHEVEUX
-----
AP SHAMP                52126
TRAITEMENT              34608
TENUE DE LA COIFFURE    17395
Autres                   6264
Name: MAILLE, dtype: int64
    
EAU DE TOILETTE
-----
EAUX PLAISIR NATURE    168525
EAUX MONOI              16662
EAUX EDIT NOEL ETE      16380
EAUX AUTRES                34
Nam

**Univers DOUCHE**

Après analyse, on se rend compte que les produits **"DOUCHE"** sont récurrents. Il parait pertinent de les regrouper dans un univers.

In [14]:
maille_douche_to_update = [
    'DOUCHE JARDINMONDE',
    'DOUCHE PLAISIRNATURE',
    'DOUCHE EDT NOEL',
    'DOUCHE MONOI',
    'DOUCHE EDT ETE',
    'DOUCHE HAMMAM',
    'DOUCHE FRAICHEUR VEG',
    'DOUCHE AUTRES',
    'DOUCHE HOMMES',
    'DOUCHE PARFUMS',
    'DOUCHE CULTUREBIO'
]

In [15]:
df.loc[df['MAILLE'].isin(maille_douche_to_update), 'UNIVERS'] = 'DOUCHE'

In [16]:
maille_douche = {
    'DOUCHE JARDINMONDE': 'JARDIMONDE',
    'DOUCHE PLAISIRNATURE': 'PLAISIRNATURE',
    'DOUCHE EDT NOEL': 'EDITION NOEL',
    'DOUCHE MONOI': 'MONOI',
    'DOUCHE EDT ETE': 'EDITION ETE',
    'DOUCHE HAMMAM': 'HAMMAM',
    'DOUCHE FRAICHEUR VEG': 'FRAICHEUR VEGAN',
    'DOUCHE AUTRES': 'AUTRES',
    'DOUCHE HOMMES': 'HOMMES',
    'DOUCHE PARFUMS': 'PARFUMEE',
    'DOUCHE CULTUREBIO': 'CULTURE BIO'
}

In [17]:
df['MAILLE'].replace(maille_douche, inplace=True)

In [18]:
print(df[df['UNIVERS'] == 'DOUCHE']['MAILLE'].value_counts())

JARDIMONDE         762858
PLAISIRNATURE      376849
EDITION NOEL       150214
PARFUMEE           101568
MONOI               64402
HOMMES              58037
CULTURE BIO         51642
HAMMAM              30611
EDITION ETE         26778
FRAICHEUR VEGAN     15180
AUTRES              10314
Name: MAILLE, dtype: int64


**Univers DEO**

Après analyse, on se rend compte que les produits **"DEO"** sont récurrents. Il parait pertinent de les regrouper dans un univers.

In [19]:
maille_deo_to_update = [
    'DEO JARDINMONDE',
    'DEO HOMMES',
    'DEO PARFUMS',
    'DEO CULTURE BIO',
    'DEO AUTRES'
]

In [20]:
df.loc[df['MAILLE'].isin(maille_deo_to_update), 'UNIVERS'] = 'DEO'

In [21]:
maille_deo = {
    'DEO JARDINMONDE': 'JARDIMONDE',
    'DEO HOMMES': 'HOMMES',
    'DEO PARFUMS': 'PARFUMEE',
    'DEO CULTURE BIO': 'CULTURE BIO',
    'DEO AUTRES': 'AUTRES'
}

In [22]:
df['MAILLE'].replace(maille_deo, inplace=True)

In [23]:
print(df[df['UNIVERS'] == 'DEO']['MAILLE'].value_counts())

JARDIMONDE     69757
PARFUMEE       40653
HOMMES         26506
CULTURE BIO    11549
AUTRES             8
Name: MAILLE, dtype: int64


**Univers JARDINMONDE**

In [24]:
maille_jardinmonde = {
    'DOUCHE JARDINMONDE': 'DOUCHE',
    'DEO JARDINMONDE': 'DEO' 
}

In [25]:
df['MAILLE'].replace(maille_jardinmonde, inplace=True)

In [26]:
print(df[df['UNIVERS'] == 'JARDINMONDE']['MAILLE'].value_counts())

Series([], Name: MAILLE, dtype: int64)


**Univers JEUNE ET LEVRE**

On décide de catégoriser cet univers en JEUNES au lieu de JEUNE ET LEVRE et déplacé la maille SOIN LEVRE dans l'univers LEVRES.

On décide également de raccourcir la maille en MASQUE, DEMAQUILLANT, CREME JOUR, LOTIONS, TRAIT, CREME NUIT.

In [27]:
df.loc[df['MAILLE'] == "SOIN LEVRES",'UNIVERS'] = 'LEVRES'
df.loc[df['MAILLE'] == "SOIN LEVRES",'MAILLE'] = 'SOIN'

In [28]:
maille_jeunes = {
    'MASQUE Jeunes Specifique': 'MASQUES',
    'DEMAQ Jeunes Specifique': 'DEMAQUILLANT',
    'CJOUR Jeunes Specifique': 'CREMES JOUR',
    'LOTIONS Jeunes Specifique': 'LOTIONS',
    'TRAIT Jeunes Specifique':'TRAITS',
    'CNUIT Jeunes Specifique': 'CREMES NUIT',
    'CJOUR Autres': 'CREMES JOUR',
    'DEMAQ Autres': 'DEMAQUILLANTS'
}

In [29]:
df['MAILLE'].replace(maille_jeunes, inplace=True)

In [30]:
df[df['UNIVERS'] == 'JEUNES']['MAILLE'].value_counts()

MASQUES          140993
DEMAQUILLANT     132980
CREMES JOUR      126859
LOTIONS           42412
TRAITS            33301
CREMES NUIT        4529
DEMAQUILLANTS         3
Name: MAILLE, dtype: int64

**Univers YEUX**

In [31]:
maille_yeux = {
    'YEUX Stylos':'STYLOS',
    'YEUX Crayons':'CRAYONS', 
    'YEUX Fard':'FARD', 
    'YEUX Mascara Specifique':'MASCARA',
    'YEUX Mascara SexyPulp':'MASCARA', 
    'YEUX Eyeliner':'EYELINER', 
    'YEUX Mascara Vertige':'MASCARA',
    'YEUX Sourcils':'SOURCILS', 
    'YEUX Mascara Basic':'MASCARA', 
    'YEUX Mascara Elixir':'MASCARA',
    'YEUX Autres':'AUTRES'
}

In [32]:
df['MAILLE'].replace(maille_yeux, inplace=True)

In [33]:
df[df['UNIVERS'] == 'YEUX']['MAILLE'].value_counts()

MASCARA     204957
STYLOS      151146
CRAYONS     137868
FARD        129906
EYELINER     57545
SOURCILS     24074
AUTRES           1
Name: MAILLE, dtype: int64

**Univers PLAISIRNATURE**

In [34]:
maille_plaisirnature = {
    'DOUCHE PLAISIRNATURE':'DOUCHE',
    'BAIN SEL HUILE PERLE':'BAIN'
}

In [35]:
df['MAILLE'].replace(maille_plaisirnature, inplace=True)

In [36]:
df[df['UNIVERS'] == 'PLAISIRNATURE']['MAILLE'].value_counts()

BAIN      155058
SAVONS    134426
Name: MAILLE, dtype: int64

**Univers HYDRATANT NOURRISSANT SOINS**

In [37]:
df[df['UNIVERS'] == 'HYDRATANT NOURRISSANT SOINS']['MAILLE'].value_counts().index

Index(['SOIN DES MAINS', 'HYDRA NOURRISANT', 'SOIN PIEDS ET JAMBES',
       'SOINS CORPS ANTI-AGE'],
      dtype='object')

In [38]:
maille_hydra = {
    'SOIN DES MAINS':'SOIN MAINS',
    'HYDRA NOURRISANT': 'HYDRATANT NOURISSANT',
    'SOINS CORPS ANTI-AGE':'SOIN CORPS'
}

In [39]:
df['MAILLE'].replace(maille_hydra, inplace=True)

In [40]:
df[df['UNIVERS'] == 'HYDRATANT NOURRISSANT SOINS']['MAILLE'].value_counts()

SOIN MAINS              231810
HYDRATANT NOURISSANT    162539
SOIN PIEDS ET JAMBES    108834
SOIN CORPS                9995
Name: MAILLE, dtype: int64

**Univers ONGLES**

In [41]:
df[df['UNIVERS'] == 'ONGLES']['MAILLE'].value_counts().index

Index(['ONG Vernis LUM', 'ONG AUTRES', 'ONG Vernis CN'], dtype='object')

In [42]:
maille_ongles = {
    'ONG Vernis LUM':'VERNIS',
    'ONG AUTRES':'AUTRES',
    'ONG Vernis CN':'VERNIS'
}
df['MAILLE'].replace(maille_ongles, inplace=True)

In [43]:
df[df['UNIVERS'] == 'ONGLES']['MAILLE'].value_counts()

VERNIS    356649
AUTRES     99021
Name: MAILLE, dtype: int64

**Univers ANTI-AGE ANTI-RIDE**

In [44]:
df[df['UNIVERS'] == 'ANTI-AGE ANTI-RIDES']['MAILLE'].value_counts().index

Index(['CJOUR AAAR', 'DEMAQ AAAR', 'CNUIT AAAR', 'TRAIT AAAR', 'LOTIONS AAAR',
       'MASQUE AAAR'],
      dtype='object')

In [45]:
maille_aaar = {
    'CJOUR AAAR':'CREMES JOUR',
    'DEMAQ AAAR':'DEMAQUILLANTS',
    'CNUIT AAAR':'CREMES NUIT',
    'TRAIT AAAR':'TRAITS', 
    'LOTIONS AAAR':'LOTIONS',
    'MASQUE AAAR':'MASQUES'
}

In [46]:
df['MAILLE'].replace(maille_aaar, inplace=True)

In [47]:
df[df['UNIVERS'] == 'ANTI-AGE ANTI-RIDES']['MAILLE'].value_counts()

CREMES JOUR      182740
DEMAQUILLANTS     77767
CREMES NUIT       51388
TRAITS            48151
LOTIONS           15415
MASQUES            3348
Name: MAILLE, dtype: int64

**Univers PARFUM**

In [48]:
df[df['UNIVERS'] == 'PARFUM']['MAILLE'].value_counts().index

Index(['PARF ET EAU DE PARF', 'EAUX PARFUMS', 'EAUX DE COLOGNE'], dtype='object')

In [49]:
maille_parfum = {
    'PARF ET EAU DE PARF': 'EAUX PARFUMS'
}

In [50]:
df['MAILLE'].replace(maille_parfum, inplace=True)

In [51]:
df[df['UNIVERS'] == 'PARFUM']['MAILLE'].value_counts()

EAUX PARFUMS       269779
EAUX DE COLOGNE     87883
Name: MAILLE, dtype: int64

**Univers TEINT**

In [52]:
df[df['UNIVERS'] == 'TEINT']['MAILLE'].value_counts().index

Index(['TEINT Font teint', 'TEINT Poudres', 'TEINT Perfect Correcteur',
       'TEINT Fard a joues'],
      dtype='object')

In [53]:
maille_teint = {
    'TEINT Font teint': 'FONT DE TEINT',
    'TEINT Poudres': 'POUDRES',
    'TEINT Perfect Correcteur': 'CORRECTEUR',
    'TEINT Fard a joues': 'FARD A JOUES'
}

In [54]:
df['MAILLE'].replace(maille_teint, inplace=True)

In [55]:
df[df['UNIVERS'] == 'TEINT']['MAILLE'].value_counts()

FONT DE TEINT    113721
POUDRES           95276
CORRECTEUR        54321
FARD A JOUES      30982
Name: MAILLE, dtype: int64

**Univers LEVRES**

In [56]:
df[df['UNIVERS'] == 'LEVRES']['MAILLE'].value_counts().index

Index(['SOIN', 'LEV RAL Lum4', 'LEV BRILLANT', 'LEV RAL Brillance',
       'LEV RAL Hydr', 'LEV RAL GrdRouge', 'LEV AUTRES', 'LEV RAL Autres'],
      dtype='object')

In [57]:
maille_levres = {
    'LEV RAL Lum4': 'ROUGE À LÈVRES',
    'LEV BRILLANT': 'BRILLANT À LÈVRES',
    'LEV RAL Brillance': 'ROUGE À LÈVRES',
    'LEV RAL Hydr': 'ROUGE À LÈVRES',
    'LEV RAL GrdRouge': 'ROUGE À LÈVRES',
    'LEV AUTRES': 'AUTRES',
    'LEV RAL Autres': 'ROUGE À LÈVRES',
}

In [58]:
df['MAILLE'].replace(maille_levres, inplace=True)

In [59]:
df[df['UNIVERS'] == 'LEVRES']['MAILLE'].value_counts()

SOIN                 238731
ROUGE À LÈVRES       159913
BRILLANT À LÈVRES     45155
AUTRES                27952
Name: MAILLE, dtype: int64

In [60]:
df[df['UNIVERS'] == 'AUTRES']['MAILLE'].value_counts().index

Index(['DEMAQ BLEUET', 'CNUIT Autres', 'LOTIONS Autres'], dtype='object')

In [61]:
maille_autres = {
    'DEMAQ BLEUET': 'DEMAQUILLANT'
}

In [62]:
df['MAILLE'].replace(maille_autres, inplace=True)

On supprime les mailles qui sont difficilement classables et qui ne resprésentent qu'une infime partie dans la données. ( < 100 lignes )

In [63]:
df = df[df['MAILLE'] != 'CNUIT Autres']
df = df[df['MAILLE'] != 'LOTIONS Autres']

In [64]:
df[df['UNIVERS'] == 'AUTRES']['MAILLE'].value_counts()

DEMAQUILLANT    175956
Name: MAILLE, dtype: int64

**Univers EAU DE TOILETTE**

In [65]:
df[df['UNIVERS'] == 'EAU DE TOILETTE']['MAILLE'].value_counts().index

Index(['EAUX PLAISIR NATURE', 'EAUX MONOI', 'EAUX EDIT NOEL ETE',
       'EAUX AUTRES'],
      dtype='object')

In [66]:
maille_eau_toilette = {
    'EAUX PLAISIR NATURE': 'PLAISIRNATURE',
    'EAUX MONOI': 'MONOI',
    'EAUX EDIT NOEL ETE': 'EDITIONS SPECIALES'
}

In [67]:
df['MAILLE'].replace(maille_eau_toilette, inplace=True)

On supprime la maille EAUX AUTRES qui n'est pas explicite et qui ne représente qu'une infime partie du dataset. ( 34 lignes )

In [68]:
df = df[df['MAILLE'] != 'EAUX AUTRES']

In [69]:
df[df['UNIVERS'] == 'EAU DE TOILETTE']['MAILLE'].value_counts()

PLAISIRNATURE         168525
MONOI                  16662
EDITIONS SPECIALES     16380
Name: MAILLE, dtype: int64

**Univers SHAMPOING**

In [70]:
df[df['UNIVERS'] == 'SHAMPOING']['MAILLE'].value_counts().index

Index(['SHAMP SPECIFIQUE', 'SHAMP TSCHEVEUX', 'SHAMP TRAITANT'], dtype='object')

In [71]:
maille_shampoing = {
    'SHAMP SPECIFIQUE': 'SPECIFIQUE',
    'SHAMP TSCHEVEUX': 'TOUS CHEVEUX',
    'SHAMP TRAITANT': 'TRAITANT'
}

In [72]:
df['MAILLE'].replace(maille_shampoing, inplace=True)

In [73]:
df[df['UNIVERS'] == 'SHAMPOING']['MAILLE'].value_counts()

SPECIFIQUE      69457
TOUS CHEVEUX    68820
TRAITANT        58959
Name: MAILLE, dtype: int64

**Univers LAIT HUILE**

In [74]:
df[df['UNIVERS'] == 'LAIT HUILE']['MAILLE'].value_counts().index

Index(['LAIT HUILE PLAISIRNATURE', 'LAIT HUILE PARFUMS', 'LAIT HUILE AUTRES',
       'EAUX CORPS AUTRES'],
      dtype='object')

In [75]:
maille_huile = {
    'LAIT HUILE PLAISIRNATURE': 'PLAISIRNATURE',
    'LAIT HUILE PARFUMS': 'PARFUMEES',
    'LAIT HUILE AUTRES': 'AUTRES',
    'EAUX CORPS AUTRES': 'EAUX CORPS'
}

In [76]:
df['MAILLE'].replace(maille_huile, inplace=True)

In [77]:
df[df['UNIVERS'] == 'LAIT HUILE']['MAILLE'].value_counts()

PLAISIRNATURE    113045
PARFUMEES         49314
AUTRES            26727
EAUX CORPS         4320
Name: MAILLE, dtype: int64

**Univers MONOI ET EDITIONS SPECIALES**

In [78]:
df[df['UNIVERS'] == 'MONOI ET EDITIONS SPECIALES']['MAILLE'].value_counts().index

Index(['LAIT HUILE MONOI', 'EAUX CORPS MONOI'], dtype='object')

In [79]:
maille_monoi_spec = {
    'DOUCHE EDT NOEL': 'DOUCHE EDITION NOEL',
    'DOUCHE EDT ETE': 'DOUCHE EDITION ETE'
}

In [80]:
df['MAILLE'].replace(maille_monoi_spec, inplace=True)

In [81]:
df[df['UNIVERS'] == 'MONOI ET EDITIONS SPECIALES']['MAILLE'].value_counts()

LAIT HUILE MONOI    52991
EAUX CORPS MONOI    20588
Name: MAILLE, dtype: int64

**Univers HOMMES**

In [82]:
df[df['UNIVERS'] == 'HOMMES']['MAILLE'].value_counts().index

Index(['EAUX HOMMES', 'SOIN HOMMES', 'APRES RASAGE', 'RASAGE'], dtype='object')

In [83]:
maille_hommes = {
    'DOUCHE HOMMES': 'DOUCHE',
    'EAUX HOMMES': 'EAUX',
    'SOIN HOMMES': 'SOIN',
    'DEO HOMMES': 'DEO'
}

In [84]:
df['MAILLE'].replace(maille_hommes, inplace=True)

In [85]:
df[df['UNIVERS'] == 'HOMMES']['MAILLE'].value_counts()

EAUX            56550
SOIN            30337
APRES RASAGE    12274
RASAGE           8667
Name: MAILLE, dtype: int64

**Univers PARFUMEE**

In [86]:
df[df['UNIVERS'] == 'PARFUMEE']['MAILLE'].value_counts().index

Index([], dtype='object')

In [87]:
maille_solaire = {
    'DOUCHE PARFUMS': 'DOUCHE',
    'DEO PARFUMS': 'DEO'
}

In [88]:
df['MAILLE'].replace(maille_solaire, inplace=True)

In [89]:
df[df['UNIVERS'] == 'PARFUMEE']['MAILLE'].value_counts()

Series([], Name: MAILLE, dtype: int64)

**Univers AUTRES**

In [90]:
df[df['UNIVERS'] == 'AUTRES']['MAILLE'].value_counts().index

Index(['DEMAQUILLANT'], dtype='object')

In [91]:
maille_autres = {
    'DEMAQ BLEUET': 'DEMAQUILLANT'
}

In [92]:
df['MAILLE'].replace(maille_autres, inplace=True)

On supprime les mailles qui sont difficilement classables et qui ne resprésentent qu'une infime partie dans la données. ( < 100 lignes )

In [93]:
df = df[df['MAILLE'] != 'CNUIT Autres']
df = df[df['MAILLE'] != 'LOTIONS Autres']

In [94]:
df[df['UNIVERS'] == 'AUTRES']['MAILLE'].value_counts()

DEMAQUILLANT    175956
Name: MAILLE, dtype: int64

**Univers CHEVEUX**

In [95]:
df[df['UNIVERS'] == 'CHEVEUX']['MAILLE'].value_counts().index

Index(['AP SHAMP', 'TRAITEMENT', 'TENUE DE LA COIFFURE', 'Autres'], dtype='object')

In [96]:
maille_cheveux = {
    'AP SHAMP': 'APRES SHAMPOING',
    'Autres': 'AUTRES'
}

In [97]:
df['MAILLE'].replace(maille_cheveux, inplace=True)

In [98]:
df[df['UNIVERS'] == 'CHEVEUX']['MAILLE'].value_counts()

APRES SHAMPOING         52126
TRAITEMENT              34608
TENUE DE LA COIFFURE    17395
AUTRES                   6264
Name: MAILLE, dtype: int64

**Univers NETTOYANT**

In [99]:
df[df['UNIVERS'] == 'NETTOYANT']['MAILLE'].value_counts().index

Index(['DEMAQ PUR', 'CJOUR PUR', 'MASQUE PUR'], dtype='object')

In [100]:
maille_pur = {
    'DEMAQ PUR': 'DEMAQUILLANT',
    'CJOUR PUR': 'CREMES DE JOUR',
    'MASQUE PUR': 'MASQUE'
}

In [101]:
df['MAILLE'].replace(maille_pur, inplace=True)

In [102]:
df[df['UNIVERS'] == 'NETTOYANT']['MAILLE'].value_counts()

DEMAQUILLANT      120622
CREMES DE JOUR     41445
MASQUE             17590
Name: MAILLE, dtype: int64

Le démaquillant de l'univers AUTRES peut être ajouté dans cette univers.

In [103]:
df.loc[(df['UNIVERS'] == "AUTRES") &(df['MAILLE'] == "DEMAQUILLANT"), 'UNIVERS'] = 'NETTOYANT'

**Univers BIO**

In [104]:
df[df['UNIVERS'] == 'BIO']['MAILLE'].value_counts().index

Index(['CJOUR BIO', 'DEMAQ BIO', 'CNUIT BIO', 'TRAIT BIO', 'LOTIONS BIO'], dtype='object')

In [105]:
maille_spa = {
    'DOUCHE CULTUREBIO': 'DOUCHE CULTUREBIO',
    'DEO CULTURE BIO': 'DEO CULTUREBIO',
    'CJOUR BIO': 'CREME DE JOUR',
    'DEMAQ BIO': 'DEMAQUILLANT',
    'CNUIT BIO': 'CREME DE NUIT',
    'TRAIT BIO': 'TRAITEMENT',
    'LOTIONS BIO': 'LOTIONS'
}

In [106]:
df['MAILLE'].replace(maille_spa, inplace=True)

In [107]:
df[df['UNIVERS'] == 'BIO']['MAILLE'].value_counts()

CREME DE JOUR    6460
DEMAQUILLANT     2255
CREME DE NUIT     394
TRAITEMENT        325
LOTIONS             1
Name: MAILLE, dtype: int64

In [108]:
univers = df['UNIVERS'].value_counts().sort_index().index
for uni in univers:
    print(uni)
    print('-----')
    print(df[df['UNIVERS'] == uni]['MAILLE'].value_counts())
    print('    ')

ANTI-AGE ANTI-RIDES
-----
CREMES JOUR      182740
DEMAQUILLANTS     77767
CREMES NUIT       51388
TRAITS            48151
LOTIONS           15415
MASQUES            3348
Name: MAILLE, dtype: int64
    
BIO
-----
CREME DE JOUR    6460
DEMAQUILLANT     2255
CREME DE NUIT     394
TRAITEMENT        325
LOTIONS             1
Name: MAILLE, dtype: int64
    
CHEVEUX
-----
APRES SHAMPOING         52126
TRAITEMENT              34608
TENUE DE LA COIFFURE    17395
AUTRES                   6264
Name: MAILLE, dtype: int64
    
DEO
-----
JARDIMONDE     69757
PARFUMEE       40653
HOMMES         26506
CULTURE BIO    11549
AUTRES             8
Name: MAILLE, dtype: int64
    
DOUCHE
-----
JARDIMONDE         762858
PLAISIRNATURE      376849
EDITION NOEL       150214
PARFUMEE           101568
MONOI               64402
HOMMES              58037
CULTURE BIO         51642
HAMMAM              30611
EDITION ETE         26778
FRAICHEUR VEGAN     15180
AUTRES              10314
Name: MAILLE, dtype: int64
    
EA